In [2]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import cupy
import nodeleys as ndl
import sys
import copy
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)

  # print(pending_for_deletion)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl

2024-10-02 23:24:11.258357: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 23:24:11.459011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 23:24:11.535173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 23:24:11.560274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 23:24:11.714398: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
import time

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,516)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,516)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(516,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(516,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(gamma0, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_gamma0 + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
loss.adic.begin_backprop()
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
w0.get_gradient()

In [ ]:
grads[0]

In [20]:
img = ndl.Node(cupy.random.uniform(high=100, sizfe=(2, 2, 3, 3)), 'img')
flatten =  node_flatten(img, 'flatten')

In [ ]:
img.tensor

In [ ]:
flatten.tensor

In [ ]:
w0.get_gradient()

In [ ]:
grads[0]

# Explore Lab

In [ ]:
tf.sqrt(784.)

In [10]:
import cupy

a = cupy.array([[
  [1,2,3],
  [4,5,6],
  [7,8,9]
],[
  [7,8,9],
  [10,11,12],
  [0,1,2]
]])

cupy.max(a, axis=[-2,-1])

array([ 9, 12])

In [8]:
data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,784*2)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,784*2)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

with tf.GradientTape(persistent=True) as g:
  tf_img = tf.Variable(tf.random.normal(shape=(1516, 2, 28, 28), dtype=tf.float64))
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_flatten = tf.reshape(tf_img, shape=(tf_img.shape[0], -1))
  # tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_flatten + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  

In [ ]:
tf_flatten[0,900]

In [ ]:
tf_img[0,1,4,4]

In [ ]:
tf.reshape(g.gradient(tf_loss, tf_flatten), shape=tf_img.shape)

In [ ]:
g.gradient(tf_loss, tf_img)

In [ ]:
g.gradient(tf_loss, tf_img)[0,:,:,0]

# Explore Lab 2

In [ ]:
import time

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
img = ndl.Node(cupy.random.normal(size=(1516,2,28,28)), name='img')

data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,784*2)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,784*2)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

flatten = node_flatten(img, 'flatten')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(flatten, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_img = tf.Variable(tf.convert_to_tensor(img.tensor.get()))
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  # tf_gamma0 = tf_data_x @ tf_w0
  tf_flatten = tf.reshape(tf_img, shape=(tf_img.shape[0], -1))
  tf_y0 = tf_flatten + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
loss.adic.begin_backprop()
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
g.gradient(tf_loss, tf_flatten)

In [ ]:
flatten.get_gradient()

# Exploration: Numpy for Convolution

In [1]:
import numpy as np

a = np.array([[0,  1,  2,  3,  4],
              [ 5,  6,  7,  8,  9],
              [10, 11, 12, 13, 14],
              [15, 16, 17, 18, 19],
              [20, 21, 22, 23, 24]], dtype=np.float64)

sub_shape = (3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides

sub_matrices = np.lib.stride_tricks.as_strided(a,view_shape,strides)

In [5]:
a = np.random.uniform(size=(20,3,6,6), high=100).astype(int)
sub_shape = (20,3,3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides
sub_matrices = np.lib.stride_tricks.as_strided(a, view_shape[2:], strides[2:])

In [ ]:
a.shape

In [ ]:
kernel_w = 3
stride_w = 1

w = 1 + int((a.shape[2] - kernel_w)/stride_w)


In [ ]:
view_shape

In [ ]:
a[0]

In [ ]:
sub_matrices.shape

In [ ]:
sub_matrices[1,0,0]

In [ ]:
view_shape

In [ ]:
strides

In [56]:


new_width = 1 + int((domain_width - kernel_width)/stride_width)
new_height = 1 + int((domain_height - kernel_height)/stride_height)

pick_shape = (new_height, new_width, batch_size, channel, kernel_height, kernel_width)

block_strides = cupy.array((block.strides + block.strides)[2:])
block_stride_strides = cupy.array((block.strides[2]*(stride_height-1), block.strides[3]*(stride_width-1), 0, 0,0,0))

memory_skip = (block_strides+block_stride_strides).get()

sub_block = cupy.lib.stride_tricks.as_strided(block, pick_shape, memory_skip)

In [ ]:
print(block[0])

In [ ]:
print(sub_block[1,1,0])

# Application

In [2]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import cupy
import nodeleys as ndl
import sys
import copy
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)

  # print(pending_for_deletion)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl

In [13]:
batch_size = 20
channel = 3

kernel_width = 3
kernel_height = 3

domain_width = 28
domain_height = 28

stride_width = 2
stride_height = 2

block = cupy.random.uniform(size=(batch_size, channel, domain_width, domain_height), high=100).astype(int)
kernel = cupy.random.uniform(size=(5, channel, kernel_width, kernel_height), high = 10).astype(int)

In [17]:
ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

blocks = ndl.Node(block, name='blocks')
kernels = ndl.Node(kernel, name='kernels')

conv2d = node_conv2d(blocks, kernels, (1,1), 'conv2d')
maxpool2d = node_maxpool2d(conv2d, pool_size=(3,2), strides=(2,2), name='maxpool2d')
flatten = node_flatten(maxpool2d, 'flatten')

In [362]:
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

# dataset
img = ndl.Node(cupy.random.normal(scale=3.0, size=(100, 3, 28, 28)), name='img')
tf_img = tfct(img)

data_y = ndl.Node(cupy.random.normal(scale=10., size=(100, 1)), name='data_y')
tf_data_y = tfct(data_y)

# config
kernels = ndl.Node(cupy.random.normal(size=(2, 3, 3, 3)), name= 'kernel', is_trainable=True)
tf_kernels = tfct(kernels)
W1 = ndl.Node(cupy.random.normal(size=(1352, 500)), name='W1', is_trainable=True)
tf_W1 = tfct(W1)
W2 = ndl.Node(cupy.random.normal(size=(500, 32)), name='W2', is_trainable=True)
tf_W2 = tfct(W2)
W3 = ndl.Node(cupy.random.normal(size=(32, 1)), name='W3', is_trainable=True)
tf_W3 = tfct(W3)

# conv2d
conv2d = node_conv2d(img, kernels, (1,1), name='conv2d')

# flatten
flatten = node_flatten(conv2d, name='flatten')

# dense
dense1 = node_matmul(flatten, W1, name='dense1') # 1352 -> 500
dense2 = node_matmul(dense1, W2, name='dense2') # 500 -> 32
dense3 = node_matmul(dense2, W3, name='dense3') # 32 -> 1

delta = node_sub(dense3, data_y, name='delta')
L = node_redsum(delta, axis=0, name='redsum')
tf_L = tfct(L)

L.adic.set_as_objective()
L.adic.begin_backprop()


([Duplet(delta, redsum; redsum),
  Triplet(dense3, data_y, delta; -),
  Triplet(dense2, W3, dense3; @),
  Triplet(dense1, W2, dense2; @),
  Triplet(flatten, W1, dense1; @),
  Duplet(conv2d, flatten; flatten),
  Triplet(img, kernel, conv2d; conv2d)],
 [(Duplet(delta, redsum; redsum), Triplet(dense3, data_y, delta; -)),
  (Triplet(dense3, data_y, delta; -), Triplet(dense2, W3, dense3; @)),
  (Triplet(dense2, W3, dense3; @), Triplet(dense1, W2, dense2; @)),
  (Triplet(dense1, W2, dense2; @), Triplet(flatten, W1, dense1; @)),
  (Triplet(flatten, W1, dense1; @), Duplet(conv2d, flatten; flatten)),
  (Duplet(conv2d, flatten; flatten), Triplet(img, kernel, conv2d; conv2d))])

In [363]:
with tf.GradientTape(persistent=True) as g:
  tf_kernels = tf.Variable(tf_kernels)
  tf_img = tf.Variable(tf_img)
  tf_conv2d = tf.nn.conv2d(tf_img, tf.transpose(tf_kernels, perm=[2,3,1,0]), strides=[1,1], padding='VALID', data_format='NCHW')
  tf_flatten = tf.keras.layers.Flatten(dtype=tf.float64)(tf_conv2d)
  tf_dense1 = tf_flatten @ tf.Variable(tf_W1)
  tf_dense2 = tf_dense1 @ tf.Variable(tf_W2)
  tf_dense3 = tf_dense2 @ tf.Variable(tf_W3)
  tf_delta = tf_dense3 - tf.Variable(tf_data_y)
  tf_L = tf.math.reduce_sum(input_tensor=tf_delta, axis=0, keepdims=True)

In [364]:
# g.gradient(tf_L, tf_img)
img.get_gradient()[0]

array([[[[[ -32.87785283,  -95.22475744, -375.92324409],
          [  53.07975678,  174.45267918, -165.69072967],
          [-177.82946923,   35.95095015, -144.26366542]],

         [[ 169.91728431,  186.90538915, -128.87165162],
          [-131.81862596,  141.88304794,  178.38886057],
          [ -32.4537047 , -197.72931821,   79.68976526]],

         [[-236.38306429,  -44.32886723,  194.87588193],
          [ -67.42230548, -315.20052098, -234.08920099],
          [ 199.08893275,  199.59240977,  163.59251715]]],


        [[[ -32.87785283,  -95.22475744, -375.92324409],
          [  53.07975678,  174.45267918, -165.69072967],
          [-177.82946923,   35.95095015, -144.26366542]],

         [[ 169.91728431,  186.90538915, -128.87165162],
          [-131.81862596,  141.88304794,  178.38886057],
          [ -32.4537047 , -197.72931821,   79.68976526]],

         [[-236.38306429,  -44.32886723,  194.87588193],
          [ -67.42230548, -315.20052098, -234.08920099],
          [ 199.088

In [443]:
result = cupy.zeros(shape=img.get_gradient().shape)

n=4
n_=1
imgx = cupy.zeros(shape=img.get_gradient().shape)
imgx[0,n,:,:,0:,n_:] = img.get_gradient()[0,n,:,:,0:,:-n_]
result += imgx[0,n,0] + img.get_gradient()[0]

n=5
n_=2
imgx = cupy.zeros(shape=img.get_gradient().shape)
imgx[0,n,:,:,0:,n_:] = img.get_gradient()[0,n,:,:,0:,:-n_]
result += imgx[0,n,0]

n=6
n_=3
imgx = cupy.zeros(shape=img.get_gradient().shape)
imgx[0,n,:,:,0:,n_:] = img.get_gradient()[0,n,:,:,0:,:-n_]
result += imgx[0,n,0]

# n = 4
# imgx = cupy.zeros(shape=img.get_gradient().shape)
# imgx[0,n,:,:,0:,n:] = img.get_gradient()[0,n,:,:,0:,:-n]
# print(imgx[0,n,0])
# result += imgx[0,n,0]
result[0,3]

array([[[[  50.63369151,    2.25885729,    5.51642055],
         [ -13.45656879, -169.67742171,  270.07101287],
         [ 156.75761001, -193.47851272,  -48.49611411]],

        [[-103.76516177,  114.17448548,  -23.6716292 ],
         [ 115.80287082, -275.85595085,  -16.11403559],
         [ 110.27863933,  -76.57585819, -329.06058936]],

        [[ 148.52712725,  -94.70739821, -187.19281433],
         [  35.97545493,  170.84461217,   -4.71591068],
         [-161.01368988,  -41.97885249,  357.79061995]]],


       [[[  50.63369151,    2.25885729,    5.51642055],
         [ -13.45656879, -169.67742171,  270.07101287],
         [ 156.75761001, -193.47851272,  -48.49611411]],

        [[-103.76516177,  114.17448548,  -23.6716292 ],
         [ 115.80287082, -275.85595085,  -16.11403559],
         [ 110.27863933,  -76.57585819, -329.06058936]],

        [[ 148.52712725,  -94.70739821, -187.19281433],
         [  35.97545493,  170.84461217,   -4.71591068],
         [-161.01368988,  -41.978852

In [290]:
171.00996586 + 200.21719487

371.22716073

In [428]:
g.gradient(tf_L, tf_img)[0,0,0]

<tf.Tensor: shape=(28,), dtype=float64, numpy=
array([ -32.87785283, -200.47857699, -443.52855967,   55.23469909,
        -11.92335128,    5.51642055, -140.58568527,   25.94552271,
        -21.73503713, -765.15430253, -620.2200052 , -445.73948992,
       -104.4032876 ,  272.53403219,  491.93490144, -316.2312739 ,
        484.34536316, -409.91223934,  670.61545814, 1438.46325162,
       -225.36720175, -363.04039406, -278.41438593,   24.45522378,
         47.26270833,   43.01198269,  152.67888419,  169.2479535 ])>

In [365]:
img.get_gradient()

array([[[[ 284.80226948, -202.07113661,   15.20283192, ...,
          -419.20361103, -199.76588955,  159.15588637],
         [ -36.41401548, -267.49736542,  150.68443325, ...,
          -295.28849585, -286.50011418,   85.89192834],
         [ 279.11944431,  201.15928858,  230.83664855, ...,
           -38.538005  ,   11.7970191 ,   20.0557697 ],
         ...,
         [-285.09814885,   90.28381507,  121.74437815, ...,
            77.13214924,  140.75241021,  -10.58139088],
         [ -33.55320936,  -19.87762929,   28.11652108, ...,
           -10.24680603,   55.70572988,  -64.06433773],
         [  91.62296559, -178.37402293,   28.38987566, ...,
          -119.41097763,  -26.26317405,   95.40093148]],

        [[  82.40890138,  106.35165243,   16.42069204, ...,
             2.87180889, -233.03080696,   34.01581204],
         [ 157.48238864, -170.17119992,  276.79016763, ...,
           242.89753793, -124.85382899,  144.91440848],
         [-133.27486689,  210.18996024,  146.33540166, .

In [446]:
import numpy as np

def consecutive_overlap_sum(arrays):
    # Stack all arrays into a 2D array
    stacked = np.vstack(arrays)
    
    # Create a triangular mask to handle shifts
    length = stacked.shape[1]
    tri_mask = np.tri(stacked.shape[0], length, dtype=int)
    
    # Element-wise multiply the mask with the stacked array
    shifted_stacked = stacked * tri_mask[:, :length]
    
    # Sum along the columns to get the final result
    result = shifted_stacked.sum(axis=0)
    
    return result

# Example with 100 arrays of length 4
arrays = [np.random.randint(1, 10, 4) for _ in range(100)]
print(arrays)
result = consecutive_overlap_sum(arrays)
print(result)


[array([8, 1, 4, 6]), array([4, 1, 5, 3]), array([6, 3, 6, 8]), array([2, 5, 1, 5]), array([3, 2, 3, 6]), array([9, 7, 5, 5]), array([7, 1, 3, 8]), array([1, 4, 1, 6]), array([8, 7, 6, 2]), array([9, 4, 9, 4]), array([7, 7, 4, 9]), array([5, 2, 1, 3]), array([1, 1, 4, 4]), array([3, 3, 7, 2]), array([1, 3, 6, 9]), array([3, 8, 7, 4]), array([9, 3, 1, 5]), array([9, 8, 2, 8]), array([7, 4, 6, 4]), array([2, 9, 8, 2]), array([8, 7, 9, 3]), array([2, 8, 6, 9]), array([4, 7, 3, 2]), array([1, 3, 9, 4]), array([4, 1, 9, 8]), array([6, 9, 2, 9]), array([1, 2, 4, 7]), array([7, 3, 7, 4]), array([2, 5, 5, 7]), array([7, 5, 9, 9]), array([2, 4, 6, 8]), array([4, 4, 2, 8]), array([6, 9, 2, 6]), array([2, 2, 9, 5]), array([7, 1, 9, 2]), array([3, 2, 5, 5]), array([5, 4, 6, 3]), array([6, 6, 1, 3]), array([6, 6, 9, 6]), array([6, 8, 1, 7]), array([3, 5, 9, 6]), array([7, 5, 2, 4]), array([9, 5, 1, 7]), array([5, 9, 4, 5]), array([7, 3, 9, 4]), array([4, 9, 4, 5]), array([7, 8, 8, 6]), array([7, 5,

In [467]:
a = np.random.uniform(high=100, size=(3,3)).astype(int)
b = np.random.uniform(high=100, size=(3,3)).astype(int)
c = np.random.uniform(high=100, size=(3,3)).astype(int)

result = np.zeros(shape=(3,5))
for i in range(3):

result = np.pad(a, ((0,0), (0,2))) + np.pad(b, ((0,0),(1,1))) + np.pad(c, ((0,0),(2,0)))

print(a,'\n')
print(b,'\n')
print(c,'\n')

print(result)

[[ 5 36 34]
 [59 75 49]
 [28 99 75]] 

[[ 1 37 56]
 [12 96 78]
 [50 97 77]] 

[[90 86 69]
 [ 2 65 17]
 [47 85 37]] 

[[  5  37 161 142  69]
 [ 59  87 147 143  17]
 [ 28 149 219 162  37]]


In [468]:
import numpy as np

# Generate random 3x3 matrices
a = np.random.uniform(high=100, size=(3, 3)).astype(int)
b = np.random.uniform(high=100, size=(3, 3)).astype(int)
c = np.random.uniform(high=100, size=(3, 3)).astype(int)

def vectorized_consecutive_pad_sum(*matrices):
    # Number of matrices and their shape
    num_matrices = len(matrices)
    n, m = matrices[0].shape

    # Create a large matrix (3D) for all padded matrices
    max_padding = num_matrices - 1
    padded_matrices = np.zeros((num_matrices, n, m + max_padding))

    # Use slicing to place each matrix in the correct padded position
    matrix_array = np.array(matrices)
    row_index = np.arange(n)  # All rows
    col_index = np.arange(m)  # All columns

    # Broadcast each matrix into the larger array with the appropriate padding
    padded_matrices[:, row_index, col_index + np.arange(num_matrices)] = matrix_array

    # Sum along the first axis (over all matrices)
    result = np.sum(padded_matrices, axis=0)

    return result

# Call the function
result = vectorized_consecutive_pad_sum(a, b, c)

print("Matrix A:\n", a, "\n")
print("Matrix B:\n", b, "\n")
print("Matrix C:\n", c, "\n")
print("Resulting Sum:\n", result)


ValueError: shape mismatch: value array of shape (3,3,3) could not be broadcast to indexing result of shape (3,3)

In [40]:
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch
from nodeleys.model import NodeleysModel

# dataset
img = ndl.Node(cupy.random.normal(size=(20, 3, 28, 28)), name='img')
data_y = ndl.Node(cupy.random.normal(size=(20, 1)), name='data_y')

class Model(NodeleysModel):
  def __init__(self):
    self.kernels = ndl.Node(cupy.random.normal(size=(2, 3, 3, 3)), name= 'kernel', is_trainable=True)
    self.W1 = ndl.Node(cupy.random.normal(size=(1352, 500)), name='W1', is_trainable=True)
    self.W2 = ndl.Node(cupy.random.normal(size=(500, 32)), name='W2', is_trainable=True)
    self.W3 = ndl.Node(cupy.random.normal(size=(32, 1)), name='W3', is_trainable=True)

  def graph(self):
    conv2d = node_conv2d(img, self.kernels, (1,1), name='conv2d')
    flatten = node_flatten(conv2d, name='flatten')

    dense1 = node_matmul(flatten, self.W1, name='dense1') # 1352 -> 500
    dense2 = node_matmul(dense1, self.W2, name='dense2') # 500 -> 32
    dense3 = node_matmul(dense2, self.W3, name='dense3') # 32 -> 1

    delta = node_sub(dense3, data_y, name='delta')
    L = node_redsum(delta, axis=0, name='redsum')
    tf_L = tfct(L)

    L.adic.set_as_objective()
    L.adic.begin_backprop()


In [41]:
x = Model()

In [39]:
x.W1.get_gradient()

array([[ 237.58264799,  -74.35502534,   26.77444905, ...,   44.43783261,
          18.05772199, -109.35254152],
       [ 145.25545059,  -45.45985492,   16.36960735, ...,   27.16880822,
          11.0402951 ,  -66.85695621],
       [ 208.61416091,  -65.2889062 ,   23.50983655, ...,   39.01952117,
          15.8559413 ,  -96.01917011],
       ...,
       [ 199.60411808,  -62.46907922,   22.49444702, ...,   37.33426857,
          15.17112341,  -91.87210344],
       [-308.03921684,   96.40545708,  -34.71457358, ...,  -57.6161402 ,
         -23.41284848,  141.78169802],
       [-195.24014777,   61.10330976,  -22.00264806, ...,  -36.51802469,
         -14.8394352 ,   89.86349191]])

In [12]:
class Dense:
  def __init__(self, units):
    self.units = units
  
  def __call__(self, prev):
    

In [222]:
def tfct(node):
  return tf.convert_to_tensor(node.tensor.get(), dtype=tf.float64)

ndl = t_update(ndl)
from nodeleys.math.forward_math_func import *
from nodeleys.graph import Switch

# dataset
img = ndl.Node(cupy.random.normal(size=(20, 3, 28, 28)), name='img')
data_y = ndl.Node(cupy.random.normal(size=(20, 1)), name='data_y')

# config
kernels = ndl.Node(cupy.random.normal(size=(2, 3, 3, 3)), name= 'kernel', is_trainable=True)
W1 = ndl.Node(cupy.random.normal(size=(1352, 500)), name='W1', is_trainable=True)
W2 = ndl.Node(cupy.random.normal(size=(500, 32)), name='W2', is_trainable=True)
W3 = ndl.Node(cupy.random.normal(size=(32, 1)), name='W3', is_trainable=True)


# conv2d
conv2d = node_conv2d(img, kernels, (1,1), name='conv2d')

# flatten
flatten = node_flatten(conv2d, name='flatten')

# dense
dense1 = node_matmul(flatten, W1, name='dense1') # 1352 -> 500
dense2 = node_matmul(dense1, W2, name='dense2') # 500 -> 32
dense3 = node_matmul(dense2, W3, name='dense3') # 32 -> 1

delta = node_sub(dense3, data_y, name='delta')
L = node_redsum(delta, axis=0, name='redsum')

L.adic.set_as_objective()
L.adic.begin_backprop()


(20, 2, 26, 26)


([Duplet(delta, redsum; redsum),
  Triplet(dense3, data_y, delta; -),
  Triplet(dense2, W3, dense3; @),
  Triplet(dense1, W2, dense2; @),
  Triplet(flatten, W1, dense1; @),
  Duplet(conv2d, flatten; flatten),
  Triplet(img, kernel, conv2d; conv2d)],
 [(Duplet(delta, redsum; redsum), Triplet(dense3, data_y, delta; -)),
  (Triplet(dense3, data_y, delta; -), Triplet(dense2, W3, dense3; @)),
  (Triplet(dense2, W3, dense3; @), Triplet(dense1, W2, dense2; @)),
  (Triplet(dense1, W2, dense2; @), Triplet(flatten, W1, dense1; @)),
  (Triplet(flatten, W1, dense1; @), Duplet(conv2d, flatten; flatten)),
  (Duplet(conv2d, flatten; flatten), Triplet(img, kernel, conv2d; conv2d))])

In [28]:
class G:
  def auxer(self, mex):
    def inner():
      print(type(mex))
      print('hex')
      mex()
    return inner
  
g = G()

@g.auxer
def aa():
  print('hast')
  pass

In [29]:
aa()

<class 'function'>
hex
hast
